In [9]:
import pandas as pd
import numpy as np

In [13]:
def read_data() -> pd.DataFrame:
    a = pd.read_csv('ans-bounds.csv')
    a['LB_VALUE'] = a.obj

    b = pd.read_csv('baron-nonconcave-bounds.csv')
    b['LB_VALUE'] = b.obj

    c = pd.read_csv('baron-concave-bounds.csv')
    c['LB_VALUE'] = c.actual_tour_obj

    l = pd.read_csv('labelling-lower-bounds.csv')
    l['LB_VALUE'] = l.obj

    return pd.concat([a, b, c, l])

In [14]:
res = read_data()
bks = pd.read_csv('bks.csv')

pwl = pd.read_csv('model-based-upper-bounds.csv')
pwl = pwl[pwl.algorithm == 'integer_linear_model_LinearModelObjectiveFunction.LINEAR_APPROX_TIGHT'].copy()

cnt = pd.read_csv('model-based-upper-bounds.csv')
cnt = cnt[cnt.algorithm == 'frankwolfe'].copy()

In [15]:
def get_gap_with_bks(row: pd.Series) -> float:
    i = row.instance_basename
    bks_row = bks[bks.instance_basename == i]

    if len(bks_row) == 0:
        print(f"No entry found in BKS file for instance {i}")
        raise RuntimeError('Missing BKS value')

    ub = bks_row.iloc[0]['BKS']
    lb = row['LB_VALUE']

    if np.isnan(ub) or ub == 0:
        return 1
    else:
        return np.abs(ub - lb) / ub

def get_gap_with_pwlin(row: pd.Series) -> float:
    i = row.instance_basename
    pwl_row = pwl[pwl.instance_basename == i]

    if len(pwl_row) == 0:
        print(f"No entry found in PWL file for instance {i}")
        raise RuntimeError('Missing PWL value')

    ub = pwl_row.iloc[0]['obj_bound']
    lb = row['LB_VALUE']

    if np.isnan(ub) or ub == 0:
        return 1
    else:
        return np.abs(ub - lb) / ub

def get_gap_with_cnt(row: pd.Series) -> float:
    i = row.instance_basename
    cnt_row = cnt[cnt.instance_basename == i]

    if len(cnt_row) == 0:
        print(f"No entry found in CNT file for instance {i}")
        raise RuntimeError('Missing CNT value')

    ub = cnt_row.iloc[0]['obj']
    lb = row['LB_VALUE']

    if np.isnan(ub) or ub == 0:
        return 1
    else:
        return np.abs(ub - lb) / ub

In [16]:
res['GAP_WITH_BKS'] = res.apply(get_gap_with_bks, axis=1)
res['GAP_WITH_PWL'] = res.apply(get_gap_with_pwlin, axis=1)
res['GAP_WITH_CNT'] = res.apply(get_gap_with_cnt, axis=1)

In [17]:
res.groupby('algorithm').mean()[['GAP_WITH_BKS', 'GAP_WITH_CNT', 'time_s']]

,GAP_WITH_BKS,GAP_WITH_CNT,time_s
algorithm,,,
ans_1.01_1.00_1.00_0.80_0.40_2000_500_10000,0.013659,0.224362,346.224440
labelling_StrongLabel,0.471876,0.586745,3187.458006
nonlinear_concave_model,0.001293,0.214552,647.857600
nonlinear_model,0.202523,0.377198,1473.286406


In [18]:
print('\\begin{tabular}{ll rrr rrr rrr rrr}')
print('\\toprule')
print(' & & ', end='')
print('\\multicolumn{3}{c}{\\textsc{LB-NLModel}} & ', end='')
print('\\multicolumn{3}{c}{\\textsc{LB-NLCModel}} & ', end='')
print('\\multicolumn{3}{c}{\\textsc{LB-Label}} & ', end='')
print('\\multicolumn{3}{c}{\\textsc{LB-ANS}} \\\\')
print('\\cmidrule(lr){3-5}\\cmidrule(lr){6-8}\\cmidrule(lr){9-11}\\cmidrule(lr){12-14}')
print('\\(\\alpha^{\\text{Tsi}}\\) & \\(\\beta^{\\text{Tsi}}\\) & ', end='')
print('G\\textsubscript{1}\% & G\\textsubscript{2}\% & T (s) & ' * 3, end='')
print('G\\textsubscript{1}\% & G\\textsubscript{2}\% & T (s) \\\\')
print('\\midrule')

for alpha in sorted(res.tsiligirides_hop_alpha.unique()):
    for beta in sorted(res.tsiligirides_hop_beta.unique()):
        f = res[(res.tsiligirides_hop_alpha == alpha) & (res.tsiligirides_hop_beta == beta)]

        print(f"{alpha:.2f} & {beta} & ", end='')

        df = f[f.algorithm == 'nonlinear_model']
        print(f"{100*df['GAP_WITH_BKS'].mean():>5.2f} & ", end='')
        print(f"{100*df['GAP_WITH_CNT'].mean():.2f} & ", end='')
        print(f"{df.time_s.mean():>6.1f} & ", end='')

        df = f[f.algorithm == 'nonlinear_concave_model']
        print(f"{100*df['GAP_WITH_BKS'].mean():>5.2f} & ", end='')
        print(f"{100*df['GAP_WITH_CNT'].mean():.2f} & ", end='')
        print(f"{df.time_s.mean():>6.1f} & ", end='')

        df = f[f.algorithm == 'labelling_StrongLabel']
        print(f"{100*df['GAP_WITH_BKS'].mean():>5.2f} & ", end='')
        print(f"{100*df['GAP_WITH_CNT'].mean():.2f} & ", end='')
        print(f"{df.time_s.mean():.1f} & ", end='')

        df = f[f.algorithm.str.contains('ans')]
        print(f"{100*df['GAP_WITH_BKS'].mean():>5.2f} & ", end='')
        print(f"{100*df['GAP_WITH_CNT'].mean():.2f} & ", end='')
        print(f"{df.time_s.mean():.1f} \\\\")

print('\\midrule')
print('\\multicolumn{2}{l}{Overall} & ', end='')

df = res[res.algorithm == 'nonlinear_model']
print(f"{100*df['GAP_WITH_BKS'].mean():>5.2f} & ", end='')
print(f"{100*df['GAP_WITH_CNT'].mean():.2f} & ", end='')
print(f"{df.time_s.mean():>6.1f} & ", end='')

df = res[res.algorithm == 'nonlinear_concave_model']
print(f"{100*df['GAP_WITH_BKS'].mean():>5.2f} & ", end='')
print(f"{100*df['GAP_WITH_CNT'].mean():.2f} & ", end='')
print(f"{df.time_s.mean():>6.1f} & ", end='')

df = res[res.algorithm == 'labelling_StrongLabel']
print(f"{100*df['GAP_WITH_BKS'].mean():>5.2f} & ", end='')
print(f"{100*df['GAP_WITH_CNT'].mean():.2f} & ", end='')
print(f"{df.time_s.mean():.1f} & ", end='')

df = res[res.algorithm.str.contains('ans')]
print(f"{100*df['GAP_WITH_BKS'].mean():>5.2f} & ", end='')
print(f"{100*df['GAP_WITH_CNT'].mean():.2f} & ", end='')
print(f"{df.time_s.mean():.1f} \\\\")

print('\\bottomrule')
print('\\end{tabular}')

\begin{tabular}{ll rrr rrr rrr rrr}
\toprule
 & & \multicolumn{3}{c}{\textsc{LB-NLModel}} & \multicolumn{3}{c}{\textsc{LB-NLCModel}} & \multicolumn{3}{c}{\textsc{LB-Label}} & \multicolumn{3}{c}{\textsc{LB-ANS}} \\
\cmidrule(lr){3-4}\cmidrule(lr){5-6}\cmidrule(lr){7-9}\cmidrule(lr){10-12}\cmidrule(lr){13-15}\cmidrule(lr){16-18}
\(\alpha^{\text{Tsi}}\) & \(\beta^{\text{Tsi}}\) & G\textsubscript{1}\% & G\textsubscript{2}\% & T (s) & G\textsubscript{1}\% & G\textsubscript{2}\% & T (s) & G\textsubscript{1}\% & G\textsubscript{2}\% & T (s) & G\textsubscript{1}\% & G\textsubscript{2}\% & T (s) \\
\midrule
0.10 & 2 &  0.00 & 13.32 &  118.5 &  0.00 & 13.32 &  102.1 & 47.14 & 53.42 & 3177.7 &  1.70 & 14.79 & 359.1 \\
0.10 & 3 &  0.09 & 19.78 &  188.4 &  0.00 & 19.70 &  129.5 & 45.47 & 55.58 & 3188.4 &  0.68 & 20.24 & 347.0 \\
0.10 & 4 &  0.00 & 22.80 &  229.5 &  0.00 & 22.80 &  155.8 & 45.08 & 57.17 & 3182.6 &  1.60 & 23.99 & 323.9 \\
0.10 & 5 &  0.00 & 21.76 &  413.6 &  0.00 & 21.76 &  192.8 & 